In [ ]:
import pandas as pd
from langchain_community.document_loaders import CSVLoader, PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Milvus
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_gigachat import GigaChat
from langchain_community.retrievers import BM25Retriever
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

def create_milvus_langchain_agent(agent_name: str, system_prompt: str, file_paths: list):
    # Загрузка и обработка данных из файлов
    documents = []
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)  # Разбиение текста на чанки

    for file_path in file_paths:
        if file_path.endswith('.csv'):
            loader = CSVLoader(file_path, encoding='utf-8')
            documents.extend(loader.load())
        elif file_path.endswith('.xlsx'):
            df = pd.read_excel(file_path)
            # Предположим, что данные в формате вопрос-ответ
            for _, row in df.iterrows():
                documents.append(Document(
                    page_content=f"Вопрос: {row['Вопрос из БЗ']}\nОтвет: {row['Ответ из БЗ']}",
                    metadata={"source": file_path}
                ))
        elif file_path.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
            pdf_docs = loader.load()
            # Разбиваем текст на чанки
            split_docs = text_splitter.split_documents(pdf_docs)
            documents.extend(split_docs)
        else:
            raise ValueError("Неподдерживаемый формат файла. Поддерживаются только CSV, Excel и PDF файлы.")

    # Инициализация модели для эмбеддингов
    embeddings = HuggingFaceEmbeddings(model_name='deepvk/USER-bge-m3', cache_folder='../cache/embedding/')

    # Создание векторной базы данных Milvus
    vector_db = Milvus.from_documents(
        documents,
        embeddings,
        connection_args={"host": "localhost", "port": "19530"},
        collection_name=agent_name
    )

    # Создание BM25Retriever
    bm25_retriever = BM25Retriever.from_documents(documents, k=3)  # k — количество возвращаемых документов

    # Создание цепочки Langchain
    template = system_prompt + "\n\nКонтекст:\n{context}\n\nВопрос: {question}\n\nОтвет:"
    prompt = ChatPromptTemplate.from_template(template)

    # Инициализация GigaChat LLM
    llm = GigaChat(credentials="your_gigachat_credentials_here")

    # Функция для объединения результатов из двух ретриверов
    def combine_retrievers(query):
        # Получаем результаты от BM25 и Milvus
        bm25_results = bm25_retriever.get_relevant_documents(query)
        milvus_results = vector_db.as_retriever().get_relevant_documents(query)
        
        # Объединяем результаты и убираем дубликаты
        combined_results = bm25_results + milvus_results
        unique_results = list({doc.page_content: doc for doc in combined_results}.values())
        
        return unique_results

    # Создание цепочки
    chain = (
        {"context": combine_retrievers, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    return chain

# Пример использования функции
agent_name = "RUTUBE_Assistant"
system_prompt = "Ты интеллектуальный помощник компании RUTUBE. Отвечай на вопросы, используя предоставленный контекст."
file_paths = ["../data/01_База_знаний.xlsx", "../data/02_Реальные_кейсы.xlsx", "../data/some_document.pdf"]

chain = create_milvus_langchain_agent(agent_name, system_prompt, file_paths)

# Тестирование цепочки
response = chain.invoke("Как подключить монетизацию на RUTUBE?")
print(response)